In [1]:
!pip install azureml.core==1.18.0

  Using cached azureml_core-1.18.0-py3-none-any.whl (2.1 MB)
ERROR: azureml-widgets 1.19.0 has requirement azureml-core~=1.19.0, but you'll have azureml-core 1.18.0 which is incompatible.
ERROR: azureml-train-core 1.19.0 has requirement azureml-core~=1.19.0, but you'll have azureml-core 1.18.0 which is incompatible.
ERROR: azureml-train-automl-runtime 1.19.0 has requirement azureml-core~=1.19.0, but you'll have azureml-core 1.18.0 which is incompatible.
ERROR: azureml-train-automl-client 1.19.0 has requirement azureml-core~=1.19.0, but you'll have azureml-core 1.18.0 which is incompatible.
ERROR: azureml-tensorboard 1.19.0 has requirement azureml-core~=1.19.0, but you'll have azureml-core 1.18.0 which is incompatible.
ERROR: azureml-telemetry 1.19.0 has requirement azureml-core~=1.19.0, but you'll have azureml-core 1.18.0 which is incompatible.
ERROR: azureml-sdk 1.19.0 has requirement azureml-core~=1.19.0, but you'll have azureml-core 1.18.0 which is incompatible.
ERROR: azureml-pipel

# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [32]:
import azureml.core
from azureml.core import Workspace
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails



In [33]:
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

# choose a name for experiment
experiment_name = 'auto-ml'
experiment=Experiment(ws, experiment_name)

Ready to use Azure ML 1.18.0 to work with capstone


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [34]:
from azureml.core import Dataset
from sklearn.model_selection import train_test_split
default_ds = ws.get_default_datastore()
ds = ws.datasets.get("heart_failure_clinical_records_dataset1")
train_ds, test_ds = ds.random_split(percentage=0.7, seed=123)
#df=ds.to_pandas_dataframe()

print("Data ready!")

Data ready!


In [35]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "capstone-cluster"

try:
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=4)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.


In [6]:
from azureml.train.automl import AutoMLConfig

automl_settings = {
    "experiment_timeout_hours" : 0.5,
    "enable_early_stopping" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 4,
    "max_cores_per_iteration": -1,
    #"n_cross_validations": 2,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=cluster_name,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='DEATH_EVENT',
                             experiment_exit_score = 0.9984,
                             enable_onnx_compatible_models=True,
                            **automl_settings
                             )

print("Ready for Auto ML run.")



Ready for Auto ML run.


In [12]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = False)


Running on remote.


In [13]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
auto-ml,AutoML_91e8e8be-a4ec-45bf-915f-26b396f7fb62,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show() 

remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

********************************

{'runId': 'AutoML_91e8e8be-a4ec-45bf-915f-26b396f7fb62',
 'target': 'capstone-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-01-03T13:51:42.651302Z',
 'endTimeUtc': '2021-01-03T14:16:11.792778Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'capstone-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment","subscription_id":"f0be6260-b326-4a23-b8fe-52db15fd80b9","resource_group":"udacity-capstone","workspace_name":"capstone","region":"westus2","compute_target":"capstone-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"AUC_weighted","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":null,"y_min":null,"y_max":null,"num_classe

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
Experiment(ws,name="auto-ml").get_runs()


<generator object Run._rehydrate_runs at 0x7f7612907a40>

In [27]:
best_run, fitted_model = remote_run.get_output()

In [17]:
 best_run.properties['model_name']

'AutoML91e8e8bea51'

In [18]:
### best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

Run(Experiment: auto-ml,
Id: AutoML_91e8e8be-a4ec-45bf-915f-26b396f7fb62_51,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.10368421052631578,
                                                                                                    min_weight_fraction_leaf=0.0,
  

In [19]:
#TODO: Save the best model
from azureml.core import Model

best_run.register_model(model_path='outputs/model.pkl', 
                        model_name='heart_failure_clinical_model_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']}
                       )



Model(workspace=Workspace.create(name='capstone', subscription_id='f0be6260-b326-4a23-b8fe-52db15fd80b9', resource_group='udacity-capstone'), name=heart_failure_clinical_model_automl, id=heart_failure_clinical_model_automl:4, version=4, tags={'Training context': 'Auto ML'}, properties={'AUC': '0.961560203504805', 'Accuracy': '0.8666666666666667'})

In [36]:
# list of registor model
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    print('Run Id:', model.run_id)
    print('framework:',model.model_framework)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

heart_failure_clinical_model_automl version: 4
Run Id: AutoML_91e8e8be-a4ec-45bf-915f-26b396f7fb62_51
framework: Custom
	 Training context : Auto ML
	 AUC : 0.961560203504805
	 Accuracy : 0.8666666666666667


keras-model version: 1
Run Id: HD_c5d99c1a-fdb1-4fe5-b293-9d383bca9138_10
framework: Custom


heart_failure_clinical_model_automl version: 3
Run Id: AutoML_3663ecec-5bd7-43f8-8a97-2d8caf4ec46d_30
framework: Custom
	 Training context : Auto ML
	 AUC : 0.9592990390050876
	 Accuracy : 0.8888888888888888


heart_failure_clinical_model_automl version: 2
Run Id: AutoML_93b41f4e-bfb3-4f1d-bf08-ecc81be63130_4
framework: Custom
	 Training context : Auto ML
	 AUC : 0.9542114188807236
	 Accuracy : 0.8888888888888888




In [37]:

from azureml.train.automl.run import AutoMLRun
best_run= AutoMLRun(Experiment(ws,name="auto-ml"), run_id = "AutoML_91e8e8be-a4ec-45bf-915f-26b396f7fb62_51" )

In [38]:
print(best_run)

Run(Experiment: auto-ml,
Id: AutoML_91e8e8be-a4ec-45bf-915f-26b396f7fb62_51,
Type: azureml.scriptrun,
Status: Completed)


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [39]:

#copy script from beat model

script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')


In [40]:
description = 'AutoML Model trained on bank marketing data to predict if a client will subscribe to a term deposit'
tags = None
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script=script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = tags, 
                                               description = 'service for Automl Classification')

aci_service_name = 'service-udacity'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)


service-udacity


In [41]:
aci_service.wait_for_deployment(True)
print(aci_service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running............................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [42]:
print(aci_service.state)
print(aci_service.scoring_uri)
print(aci_service.swagger_uri)
#service.get_logs()

Healthy
http://7162bd74-5909-44ec-8a0a-737c74263f9a.westus2.azurecontainer.io/score
http://7162bd74-5909-44ec-8a0a-737c74263f9a.westus2.azurecontainer.io/swagger.json


In [43]:
X_test,y_test=test_ds.to_pandas_dataframe()[['age','anaemia','creatinine_phosphokinase',
        'diabetes','ejection_fraction','high_blood_pressure',
        'platelets','serum_creatinine','serum_sodium',
        'sex','smoking','time']],df['DEATH_EVENT']

In [45]:
import json
import requests
X_test_json = X_test[10:30].to_json(orient='records')
print(X_test_json)

[{"age":94.0,"anaemia":0,"creatinine_phosphokinase":582,"diabetes":1,"ejection_fraction":38,"high_blood_pressure":1,"platelets":263358.03,"serum_creatinine":1.83,"serum_sodium":134,"sex":1,"smoking":0,"time":27},{"age":50.0,"anaemia":1,"creatinine_phosphokinase":249,"diabetes":1,"ejection_fraction":35,"high_blood_pressure":1,"platelets":319000.0,"serum_creatinine":1.0,"serum_sodium":128,"sex":0,"smoking":0,"time":28},{"age":82.0,"anaemia":1,"creatinine_phosphokinase":855,"diabetes":1,"ejection_fraction":50,"high_blood_pressure":1,"platelets":321000.0,"serum_creatinine":1.0,"serum_sodium":145,"sex":0,"smoking":0,"time":30},{"age":50.0,"anaemia":0,"creatinine_phosphokinase":582,"diabetes":1,"ejection_fraction":38,"high_blood_pressure":0,"platelets":310000.0,"serum_creatinine":1.9,"serum_sodium":135,"sex":1,"smoking":1,"time":35},{"age":42.0,"anaemia":1,"creatinine_phosphokinase":250,"diabetes":1,"ejection_fraction":15,"high_blood_pressure":0,"platelets":213000.0,"serum_creatinine":1.3,"s

TODO: In the cell below, send a request to the web service you deployed to test it.

In [46]:
data = "{\"data\": " + X_test_json +"}"
headers = {'Content-Type': 'application/json'}

resp = requests.post(aci_service.scoring_uri, data, headers=headers)

y_pred = json.loads(json.loads(resp.text))['result']
print("predict:",y_pred)

predict: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


In [47]:
print("actual: ",y_test[10:30].to_list())

actual:  [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1]


TODO: In the cell below, print the logs of the web service and delete the service

In [48]:
aci_service.delete()